In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd gdrive/MyDrive/homeWorkDataSet

/content/gdrive/MyDrive/homeWorkDataSet


در این نوت بوک سعی شده طبق روند معمول نوت بوک های پایتورچ طراحی صورت پذیرد.

هر چند بخش اعظمی از کد به صورت دستی نوشته شده و حاوی تغییرات زیادی است

به عنوان مثال از هیچ شبکه ی از پیش آماده ای استفاده نشده و با دیزاین یک شبکه در کلاسی به نام 

resnet 

با ارث بری از کلاس پایه شبکه های عصبی در پایتورچ
 سعی شده تا بهترین نتیجه گرفته شود.

مدل با پارامتر های مختلف آزمایش شده که در انتها سه نمونه از دقت های بدست آمده را مشاهده می کنید.

بهترین پارامتر ها در همین نوت بوک حاضر اعمال شده است

In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
plt.ion()

In [ ]:
from torch import Tensor
import torch.nn as nn
from torch.hub import load_state_dict_from_url
from typing import Type, Any, Callable, Union, List, Optional

data augmentation to generate more instances from training set

data preprocessing and resize all images to a size that is comfortable for gpu

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# in homeWorkDataSet Directory
data_dir = 'data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=8)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

I have preprocess all the datas and split them to two folders which names is train and val

they locates under data directory. if you want to run this notebook, tell me to give access to you.

the code I used to stratified sampling 20 percent of data and put them as val set is the below code.

In [ ]:
# import os
# from pathlib import Path
# import shutil
# import random
# val_size = 0.2
# val_dir = Path('data/val/')
# train_dir = Path('data/train/')
# for entry in train_dir.iterdir():
#   images = [f for f in entry.iterdir()]
#   size = len(images)
#   randoms = set()
#   for i in range(int(size*val_size)):
#     random_image = random.choice(images)
#     while random_image in randoms:
#       random_image = random.choice(images)
#     randoms.add(random_image)
#     dest_dir = val_dir/entry.name
#     dest_dir.mkdir(parents=True, exist_ok=True)
#     shutil.move(random_image, dest_dir/random_image.name)

In [ ]:
device

device(type='cuda', index=0)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, groups=1, bias=False, dilation=1)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, groups=1, bias=False, dilation=1)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [ ]:
class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = nn.Conv2d(inplanes, width, kernel_size=1, stride=1, bias=False)
        self.bn1 = norm_layer(width)
        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=dilation, groups=groups, bias=False, dilation=dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = nn.Conv2d(width, planes * self.expansion, kernel_size=1, stride=1, bias=False)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


I designed my own resnet. the detail of network is in _forward_impl method

In [ ]:
class ResNet(nn.Module):

    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None
    ) -> None:
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


for better results, I upload the weights of resnet18 to my network, this is an advice from Mr shahverdi

In [ ]:
model = ResNet(BasicBlock, [2, 2, 2, 2])
state_dict = load_state_dict_from_url('https://download.pytorch.org/models/resnet18-5c106cde.pth', progress=True)
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
model_ft = model
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 120)
model_ft = model_ft.to(device)
criterion = nn.CrossEntropyLoss()
# fit the parameters with sgd optimizer
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay learning rate by a factor of 0.1 every 7 epochs
# use weight decay to avoid overfitting 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25) # batch size = 16

Epoch 0/24
----------
train Loss: 2.7742 Acc: 0.3734
val Loss: 1.0996 Acc: 0.6933

Epoch 1/24
----------
train Loss: 1.6449 Acc: 0.5786
val Loss: 0.9013 Acc: 0.7331

Epoch 2/24
----------
train Loss: 1.4491 Acc: 0.6169
val Loss: 0.7797 Acc: 0.7642

Epoch 3/24
----------
train Loss: 1.3407 Acc: 0.6403
val Loss: 0.7750 Acc: 0.7652

Epoch 4/24
----------
train Loss: 1.2726 Acc: 0.6539
val Loss: 0.7997 Acc: 0.7522

Epoch 5/24
----------
train Loss: 1.2243 Acc: 0.6664
val Loss: 0.7636 Acc: 0.7660

Epoch 6/24
----------
train Loss: 1.1567 Acc: 0.6839
val Loss: 0.7640 Acc: 0.7655

Epoch 7/24
----------
train Loss: 1.0335 Acc: 0.7222
val Loss: 0.6482 Acc: 0.8077

Epoch 8/24
----------
train Loss: 0.9869 Acc: 0.7334
val Loss: 0.6487 Acc: 0.8084

Epoch 9/24
----------
train Loss: 0.9480 Acc: 0.7457
val Loss: 0.6299 Acc: 0.8170

Epoch 10/24
----------
train Loss: 0.9356 Acc: 0.7505
val Loss: 0.6310 Acc: 0.8197

Epoch 11/24
----------
train Loss: 0.9347 Acc: 0.7482
val Loss: 0.6263 Acc: 0.8139

Ep

In [ ]:
# below train was last about 2 hours and is my best accuracy.

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25) # batch size = 32, my best accuracy on val = 90.33%

Epoch 0/24
----------
train Loss: 3.0126 Acc: 0.3672
val Loss: 1.2289 Acc: 0.7182

Epoch 1/24
----------
train Loss: 1.6569 Acc: 0.6059
val Loss: 0.8176 Acc: 0.7883

Epoch 2/24
----------
train Loss: 1.3911 Acc: 0.6453
val Loss: 0.6698 Acc: 0.8173

Epoch 3/24
----------
train Loss: 1.2613 Acc: 0.6657
val Loss: 0.5930 Acc: 0.8335

Epoch 4/24
----------
train Loss: 1.1863 Acc: 0.6810
val Loss: 0.5354 Acc: 0.8459

Epoch 5/24
----------
train Loss: 1.1354 Acc: 0.6947
val Loss: 0.4823 Acc: 0.8603

Epoch 6/24
----------
train Loss: 1.0720 Acc: 0.7112
val Loss: 0.4286 Acc: 0.8751

Epoch 7/24
----------
train Loss: 0.9978 Acc: 0.7324
val Loss: 0.4012 Acc: 0.8872

Epoch 8/24
----------
train Loss: 0.9765 Acc: 0.7421
val Loss: 0.3885 Acc: 0.8928

Epoch 9/24
----------
train Loss: 0.9515 Acc: 0.7482
val Loss: 0.3787 Acc: 0.8953

Epoch 10/24
----------
train Loss: 0.9503 Acc: 0.7476
val Loss: 0.3767 Acc: 0.8960

Epoch 11/24
----------
train Loss: 0.9467 Acc: 0.7483
val Loss: 0.3691 Acc: 0.8964

Ep

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25) # batch size = 64

Epoch 0/24
----------
train Loss: 3.8615 Acc: 0.2172
val Loss: 2.3468 Acc: 0.5371

Epoch 1/24
----------
train Loss: 2.3488 Acc: 0.5177
val Loss: 1.4533 Acc: 0.6935

Epoch 2/24
----------
train Loss: 1.7978 Acc: 0.6060
val Loss: 1.1080 Acc: 0.7412

Epoch 3/24
----------
train Loss: 1.5422 Acc: 0.6421
val Loss: 0.9649 Acc: 0.7615

Epoch 4/24
----------
train Loss: 1.3869 Acc: 0.6640
val Loss: 0.8754 Acc: 0.7726

Epoch 5/24
----------
train Loss: 1.2958 Acc: 0.6776
val Loss: 0.8051 Acc: 0.7800

Epoch 6/24
----------
train Loss: 1.2200 Acc: 0.6962
val Loss: 0.7600 Acc: 0.7881

Epoch 7/24
----------
train Loss: 1.1528 Acc: 0.7143
val Loss: 0.7463 Acc: 0.7947

Epoch 8/24
----------
train Loss: 1.1415 Acc: 0.7149
val Loss: 0.7461 Acc: 0.7947

Epoch 9/24
----------
train Loss: 1.1257 Acc: 0.7157
val Loss: 0.7382 Acc: 0.7972

Epoch 10/24
----------
train Loss: 1.1281 Acc: 0.7197
val Loss: 0.7289 Acc: 0.7969

Epoch 11/24
----------
train Loss: 1.1119 Acc: 0.7246
val Loss: 0.7312 Acc: 0.8008

Ep

# My best Acc on val is : 90.33%